### Etymology

This notebook is used to web scrapping and etymology data analysis.

In [1]:
import pandas as pd
import numpy as np
import glob
import nltk
from nltk.stem import WordNetLemmatizer
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse
#nltk.download('wordnet')
#nltk.download('omw-1.4')

In [2]:
lang_folder = "Turkish"

### Etymologeek

In [ ]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/Word/Merge/Word_Merge2.xlsx")
#df_word = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Result/Word/Merge/Word_Merge2.csv")
df_word = df_word.head(10000)
df_word

In [ ]:
#word_lemma = []
#for word in df_word["word"]:
#    word_lemma.append(WordNetLemmatizer().lemmatize(word))
#    word_unique = set(word_lemma)

In [ ]:
#len(word_lemma)

In [ ]:
#len(word_unique)

In [ ]:
word_list = df_word["word"].to_list()

In [ ]:
result_list = []
#for i in word_unique:
for i in word_list:
    try:
        response = requests.get(f"https://etymologeek.com/search/all/{i}")
        soup = BeautifulSoup(response.content, 'lxml')
        word = soup.find('tbody', attrs={"id":"tb"})
        dict_word = word.findAll('td', attrs={"data-th":"Dic. entry"})
        lang = word.findAll('td', attrs={"data-th":"Language"})
        definition = word.findAll('td', attrs={"data-th":"Definition"})
        a_href = word.findAll('td', attrs={"data-th":"Dic. entry"})
        iter_values = zip(dict_word, lang, definition, a_href)
        for a, b, c, d in iter_values:
            var1 = a.get_text()
            var2 = b.get_text()  # string
            var3 = c.get_text()  # string
            var4 = d.find('a').get('href')
            var5 = f"https://etymologeek.com{var4}"
            result_list.append([i, var1, var2, var3, var5])
    except:
        pass

df_result = pd.DataFrame(result_list)
df_result.rename(columns={0:"search_word", 1:"dict_entry", 2:"language", 3:"definition", 4:"href"}, inplace=True)
df_result

In [ ]:
df_result.to_excel(f"{lang_folder.capitalize()}_Etymologeek_Main_Result.xlsx", index=False)

#### Scrapping Additional Info

In [ ]:
df_main_word = pd.read_excel(f"{lang_folder.capitalize()}_Etymologeek_Main_Result.xlsx")
df_main_word

In [ ]:
df_main_word.search_word.nunique()

In [ ]:
df_main_word.language.nunique()

In [ ]:
search_word = df_main_word["search_word"].to_list()
dict_entry_main = df_main_word["dict_entry"].to_list()
language_main = df_main_word["language"].to_list()
definition_main = df_main_word["definition"].to_list()	
href_url = df_main_word["href"].to_list()
main_iter_values = zip(search_word, dict_entry_main, language_main, definition_main, href_url)
result_list2 = []
for search, dict_ent, lang_main, def_main, href in main_iter_values:
    try:
        response = requests.get(f"{href}")
        soup = BeautifulSoup(response.content, 'lxml')
        word = soup.find('tbody', attrs={"id":"tb"})
        dict_entry = word.findAll('td', attrs={"data-th":"Dic. entry"})
        lang = word.findAll('td', attrs={"data-th":"Language"})
        definition = word.findAll('td', attrs={"data-th":"Definition"})
        iter_values = zip(dict_entry, lang, definition)
        for a, b, c in iter_values:
            var1 = a.get_text()
            var2 = b.get_text()
            var3 = c.get_text()
            result_list2.append([search, dict_ent, lang_main, def_main, var1, var2, var3])
    except:
        pass

df_result2 = pd.DataFrame(result_list2)
df_result2.rename(columns={0:"search_word", 1:"dict_entry_main", 2:"language_main", 3:"definition_main", 4:"dict_entry", 5:"language", 6:"definition"}, inplace=True)
df_result2

In [ ]:
df_result2.language_main = df_result2.language_main.apply(lambda x: x.strip())
df_result2.language = df_result2.language.apply(lambda x: x.strip())

In [ ]:
df_result2.to_excel(f"{lang_folder.capitalize()}_Etymologeek_Main_And_Additional_Result.xlsx", index=False)

### Github Data Process 

In [ ]:
df_github = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology.csv")
df_github

#### Select 1

In [ ]:
df_select = df_github.loc[:,["lang","term","reltype","related_lang","related_term"]]
df_select

In [ ]:
df_select.isnull().sum()

In [ ]:
df_select.drop_duplicates(inplace=True)
df_select.dropna(inplace=True)
df_select.reset_index(drop=True, inplace=True)
df_select

In [ ]:
df_select.to_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_clean.csv", index=False)

#### Select 2

In [ ]:
df_select2 = df_github.loc[:,["term_id","lang","term","reltype","related_term_id","related_lang","related_term"]]
df_select2

In [ ]:
df_select2.isnull().sum()

In [ ]:
df_select2.drop_duplicates(inplace=True)
df_select2.dropna(inplace=True)
df_select2.reset_index(drop=True, inplace=True)
df_select2

In [ ]:
df_select2.to_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_clean2.csv", index=False)

#### Sample Process

In [ ]:
df_clean = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_clean.csv")
df_clean = df_clean.head(150000)
df_clean

In [ ]:
df_clean["lang"] = df_clean["lang"].apply(lambda x: x.capitalize())
df_clean["lang"] = df_clean["lang"].apply(lambda x: x.strip())
df_clean["term"] = df_clean["term"].apply(lambda x: x.lower())
df_clean["term"] = df_clean["term"].apply(lambda x: x.strip())
df_clean["related_lang"] = df_clean["related_lang"].apply(lambda x: x.capitalize())
df_clean["related_lang"] = df_clean["related_lang"].apply(lambda x: x.strip())
df_clean["related_term"] = df_clean["related_term"].apply(lambda x: x.lower())
df_clean["related_term"] = df_clean["related_term"].apply(lambda x: x.strip()) 

In [ ]:
result_list = []
for i in range(len(df_clean)):
    lang_11 = df_clean.loc[i,"lang"]
    term_11 = df_clean.loc[i,"term"]
    reltype_11	= df_clean.loc[i,"reltype"]
    lang_12 = df_clean.loc[i,"related_lang"]
    term_12 = df_clean.loc[i,"related_term"]
    
    try:
        df_var = df_clean[(df_clean["term"] == f"{term_12}")]
        df_var.reset_index(drop=True, inplace=True)
        j = 0
        while j < len(df_var):                    
            lang_21 = df_var.loc[j,"lang"]
            term_21 = df_var.loc[j,"term"]
            #reltype_21	= df_var.loc[j,"reltype"]
            lang_22 = df_var.loc[j,"related_lang"]
            term_22 = df_var.loc[j,"related_term"]
            if ((f"{lang_21}" == f"{lang_12}") & (f"{term_21}" == f"{term_12}")):
                result_list.append([lang_11,term_11,reltype_11,lang_22,term_22])
            else:
                result_list.append([lang_11,term_11,reltype_11,lang_12,term_12])
            
            j+=1
    except:
        result_list.append([lang_11,term_11,reltype_11,lang_12,term_12])

In [ ]:
df_result = pd.DataFrame(result_list, columns=["lang","term","reltype","related_lang","related_term"])
df_result.drop_duplicates(inplace=True)
df_result.reset_index(drop=True,inplace=True)
df_result

In [ ]:
#df_result.to_csv("etymology_cross_check_sample.csv", index=False)

#### Multi Process

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# 1-Way
df_clean = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_clean.csv")
df_clean = df_clean.head(150000)
df_clean

In [ ]:
df_clean["lang"] = df_clean["lang"].apply(lambda x: x.capitalize())
df_clean["lang"] = df_clean["lang"].apply(lambda x: x.strip())
df_clean["term"] = df_clean["term"].apply(lambda x: x.lower())
df_clean["term"] = df_clean["term"].apply(lambda x: x.strip())
df_clean["related_lang"] = df_clean["related_lang"].apply(lambda x: x.capitalize())
df_clean["related_lang"] = df_clean["related_lang"].apply(lambda x: x.strip())
df_clean["related_term"] = df_clean["related_term"].apply(lambda x: x.lower())
df_clean["related_term"] = df_clean["related_term"].apply(lambda x: x.strip()) 

In [ ]:
d_list = range(len(df_clean))

In [ ]:
resultlist = []

In [ ]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def cross_check(i):
    lang_11 = df_clean.loc[i,"lang"]
    term_11 = df_clean.loc[i,"term"]
    reltype_11	= df_clean.loc[i,"reltype"]
    lang_12 = df_clean.loc[i,"related_lang"]
    term_12 = df_clean.loc[i,"related_term"]
    
    try:
        df_var = df_clean[(df_clean["term"] == f"{term_12}")]
        df_var.reset_index(drop=True, inplace=True)
        j = 0
        while j < len(df_var):                    
            lang_21 = df_var.loc[j,"lang"]
            term_21 = df_var.loc[j,"term"]
            reltype_21	= df_var.loc[j,"reltype"]
            lang_22 = df_var.loc[j,"related_lang"]
            term_22 = df_var.loc[j,"related_term"]
            if ((f"{lang_21}" == f"{lang_12}") & (f"{term_21}" == f"{term_12}")):
                resultlist.append([lang_11,term_11,reltype_21,lang_22,term_22])
            else:
                resultlist.append([lang_11,term_11,reltype_11,lang_12,term_12])
            
            j+=1
    except:
        resultlist.append([lang_11,term_11,reltype_11,lang_12,term_12])

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(cross_check, d_list) # string_word liste

In [ ]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list, columns=["lang","term","reltype","related_lang","related_term"])
df_result.drop_duplicates(inplace=True)
df_result.reset_index(drop=True,inplace=True)
df_result

In [ ]:
#df_result.to_csv("etymology_cross_check.csv", index=False)

In [ ]:
#df_result.to_excel("etymology_cross_check.xlsx", index=False)

In [ ]:
# 2-Way (Master)
df_clean2 = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_clean2.csv")
df_clean2 = df_clean2.head(150000)
df_clean2

In [ ]:
df_clean2["lang"] = df_clean2["lang"].apply(lambda x: x.capitalize())
df_clean2["lang"] = df_clean2["lang"].apply(lambda x: x.strip())
df_clean2["term_id"] = df_clean2["term_id"].apply(lambda x: x.strip())
df_clean2["related_lang"] = df_clean2["related_lang"].apply(lambda x: x.capitalize())
df_clean2["related_lang"] = df_clean2["related_lang"].apply(lambda x: x.strip())
df_clean2["related_term_id"] = df_clean2["related_term_id"].apply(lambda x: x.strip()) 

In [ ]:
d_list2 = range(len(df_clean2))

In [ ]:
resultlist2 = []

In [ ]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def cross_check(i):
    term_id_11 = df_clean2.loc[i,"term_id"]
    lang_11 = df_clean2.loc[i,"lang"]
    term_11 = df_clean2.loc[i,"term"]
    reltype_11	= df_clean2.loc[i,"reltype"]
    term_id_12 = df_clean2.loc[i,"related_term_id"]
    lang_12 = df_clean2.loc[i,"related_lang"]
    term_12 = df_clean2.loc[i,"related_term"]
    
    try:
        df_var = df_clean2[(df_clean2["term_id"] == f"{term_id_12}")]
        df_var.reset_index(drop=True, inplace=True)
        j = 0
        while j < len(df_var):
            term_id_21 = df_var.loc[j,"term_id"]
            #lang_21 = df_var.loc[j,"lang"]
            #term_21 = df_var.loc[j,"term"]
            reltype_21	= df_var.loc[j,"reltype"]
            term_id_22 = df_var.loc[j,"related_term_id"]
            lang_22 = df_var.loc[j,"related_lang"]
            term_22 = df_var.loc[j,"related_term"]
            if (f"{term_id_21}" == f"{term_id_12}"):
                resultlist2.append([term_id_11,lang_11,term_11,reltype_21,term_id_22,lang_22,term_22])
            else:
                resultlist2.append([term_id_11,lang_11,term_11,reltype_11,term_id_12,lang_12,term_12])
            
            j+=1
    except:
        resultlist2.append([term_id_11,lang_11,term_11,reltype_11,term_id_12,lang_12,term_12])

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(cross_check, d_list2) # string_word liste

In [ ]:
result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=["term_id","lang","term","reltype","related_term_id","related_lang","related_term"])
df_result2.drop_duplicates(inplace=True)
df_result2.reset_index(drop=True,inplace=True)
df_result2

In [ ]:
df_result2.to_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_cross_check2.csv", index=False)

### Etimoloji Sözlük

In [3]:
lang_folder = "Turkish"
start_num = 0
end_num = 40000

In [4]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/Word/Merge/Word_Merge2.xlsx")
df_word = df_word.iloc[start_num:end_num,]
df_word

,word,frequency
0,bir,18835735
1,fikret,12101580
2,bu,11062659
3,ne,8025880
4,ve,7766036
...,...,...
39995,ekipmanlar,974
39996,şanslılar,974
39997,aksanıyla,974
39998,yargılanacak,974


In [5]:
word_list = df_word["word"].tolist()
#word_list

In [6]:
result_list = []
#for i in word_unique:
for i in word_list:
    try:
        response = requests.get(f"https://www.etimolojiturkce.com/kelime/{i}")
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find('div', attrs={"class":"holder"})
        data_main = data.find('div', attrs={"class":"main"})
        word = data_main.find('h1').get_text()        
        all_paraf = data_main.find_all('p')        
        koken = all_paraf[3].get_text()
        span = all_paraf[3].span        
        all_b = span.find_all('b')
        all_i = span.find_all('i')  # option
        word_root = all_i[0].get_text()  # option         
        lang = all_b[0].get_text()        
        lang_root = all_b[-1].get_text()

        result_list.append([i, word, lang, word_root, koken, lang_root])  # option
        #result_list.append([i, word, lang, koken, lang_root])    
                
        #iter_values = zip(word3, koken2)
        #for a, b in iter_values:
        #    var1 = a  # span
        #    var2 = b  # string
        #   
        #    result_list.append([i, var1, var2])
    except:
        pass

df_result = pd.DataFrame(result_list)
df_result.rename(columns={0:"search_word", 1:"word", 2:"lang", 3:"word_root", 4:"koken", 5:"lang_root"}, inplace=True)  # option
#df_result.rename(columns={0:"search_word", 1:"word", 2:"lang", 3:"koken", 4:"lang_root"}, inplace=True) 
df_result

,search_word,word,lang,word_root,koken,lang_root
0,bir,Bir,Eski Türkçe,bir,Eski Türkçe bir sözcüğünden evrilmiştir.,Eski Türkçe
1,ve,Ve,Arapça,wa,"Arapça wa و z ""ve (bağlaç), - adına (edat)"" f...",Arapça
2,için,İçin,Eski Türkçe,üçün,"Eski Türkçe üçün ""nedensellik edatı"" sözcüğün...",Eski Türkçe
3,çok,Çok,Ana Türkçe,*çaw-,"Ana Türkçe yazılı örneği bulunmayan *çaw- ""ba...",Ana Türkçe
4,ama,Ama,Arapça,ammā,"Arapça ammā أمّا z ""gelgelelim, maamafih (bağ...",Arapça
...,...,...,...,...,...,...
3658,masör,Masör,Fransızca\r,masseur,"Fransızca\r masseur ""masaj yapan"" sözcüğünden...",Fransızca\r
3659,baraka,Baraka,İtalyanca\r,baracca,"İtalyanca\r baracca ""kulübe, derme çatma yapı...",İtalyanca\r
3660,oğlak,Oğlak,Eski Türkçe,ogul,Eski Türkçe ogul sözcüğünden +Ak sonekiyle tü...,Eski Türkçe
3661,boykot,Boykot,İngilizce\r,boycott,"İngilizce\r boycott ""bir mal veya hizmeti sat...",İngilizce\r


In [7]:
df_result["word"] = df_result["word"].apply(lambda x: x.lower())

In [8]:
def remove(x):
    try:
        x = x.replace("_x000D_","")
        x = x.replace("\n","")
        x = x.replace("\r","")
        x = x.strip()
    except:
        x = x
    return x

In [9]:
df_result["lang"] = df_result["lang"].apply(lambda x: remove(x))
df_result["koken"] = df_result["koken"].apply(lambda x: remove(x))
df_result["lang_root"] = df_result["lang_root"].apply(lambda x: remove(x))

In [10]:
#df_result.to_excel("Sample.xlsx", index=False)

In [11]:
df_result.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/{lang_folder.capitalize()}_{start_num}-{end_num}_Word_Etimoloji_Result.xlsx", index=False)

### Nisanyan

In [ ]:
df_word_plus_nisanyan = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/Nisanyan Words Contain Plus.xlsx")
df_word_plus_nisanyan

In [ ]:
def remove(x):
    try:
        x = x.replace("\n","")
        x = x.replace("\r","")
        x = x.strip()
    except:
        x = x
    return x

In [ ]:
df_word_plus_nisanyan["term"] = df_word_plus_nisanyan["term"].apply(lambda x: remove(x))
df_word_plus_nisanyan["related_lang"] = df_word_plus_nisanyan["related_lang"].apply(lambda x: remove(x))
df_word_plus_nisanyan["related_term"] = df_word_plus_nisanyan["related_term"].apply(lambda x: remove(x))
df_word_plus_nisanyan

In [ ]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Result/Word/Merge/Word_Merge2.xlsx")
#df_word = df_word.head(10000)
df_word

In [ ]:
disable_term_list = ["+ark","+gen","+gon","di+","+men","deka+","desi+","bis+","end(o)+","fil(o)+","mili+","ö+","or(o)+","pan(to)+","poli+","port+","post+","vaz(o)+"]

In [ ]:
set_plus_term = set(df_word_plus_nisanyan.loc[:,"term"].values.tolist())
set_disable_term = set(disable_term_list)

In [ ]:
df_term_diff = pd.DataFrame(set_plus_term.difference(set_disable_term), columns=["term"])
df_term_diff

In [ ]:
word_list = df_term_diff.loc[:,"term"].values.tolist()

In [ ]:
df_word_result = pd.DataFrame()
for i in word_list:
    if i.startswith("+"):
        j = i.lstrip("+")
        word_in_video = df_word[df_word.word.str.contains(fr"{j}(?:$)", na=True)]  # string+ext=> word
        word_in_video.insert(0,"term",i)
    elif i.endswith("+"):
        j = i.rstrip("+")
        word_in_video = df_word[df_word.word.str.contains(fr"(?:^){j}", na=True)]  # ext+string=> word
        word_in_video.insert(0,"term",i)
    else:
        pass
    #word_in_video.insert(0,"word_ext",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
#df_word_result = df_word_result.sort_values(by="frequency", ascending=False)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result 

In [ ]:
df_pre_suff_nis_merge = pd.merge(df_word_result,df_word_plus_nisanyan, how="inner", on="term")
df_pre_suff_nis_merge

In [ ]:
df_pre_suff_nis_merge.word.nunique()

In [ ]:
df_pre_suff_nis_merge.to_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/Nisanyan_Prefix_Suffix_Term_In_Word.xlsx", index=False)

#### SparkNLP Lemma

In [ ]:
df_spark_lemma = pd.read_json("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/wR3tE0i-zc6Pl8revsMq4.json")  # AWS Marketplace lemma result
df_spark_lemma = df_spark_lemma.loc[:,["lem","token"]]
df_spark_lemma.rename(columns={"token":"word"}, inplace=True)
df_spark_lemma

In [ ]:
df_spark_lemma.lem.nunique()

In [ ]:
df_pre_suff_nis_merge = pd.read_excel("Nisanyan_Prefix_Suffix_Term_In_Word.xlsx")
df_pre_suff_nis_merge

In [ ]:
df_lemma_merge = pd.merge(df_pre_suff_nis_merge,df_spark_lemma, how="inner", on="word")
df_lemma_merge.drop_duplicates(inplace=True)
df_lemma_merge

In [ ]:
df_lemma_merge.drop_duplicates(subset="lem", inplace=True)
df_lemma_merge.reset_index(drop=True, inplace=True)
df_lemma_merge

In [ ]:
df_lemma_merge.to_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/Nisanyan_Prefix_Suffix_Term_In_Word_Lemma_Process.xlsx", index=False)